In [6]:
import os
# set the device to run
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import numpy as np
from sklearn.datasets import make_classification, make_regression

from py_boost import GradientBoosting 

In [25]:
%%time
X, y = make_classification(150000, 100, n_classes=5, random_state=42, n_informative=30)
X = X.astype(np.float32)
X_test, y_test = X[:50000], y[:50000]
X, y = X[50000:], y[50000:]

sample_weight = np.random.rand(150000)
w_test = sample_weight[:50000]
w = sample_weight[50000:]

CPU times: user 3.93 s, sys: 3.36 s, total: 7.29 s
Wall time: 1.23 s


In [4]:
from py_boost.gpu.losses import *

In [5]:
%%time
model = GradientBoosting('binary', F1Score(), 
                         ntrees=1000, lr=.01, verbose=100, es=0, lambda_l2=1,
                         subsample=.8, colsample=.8, min_data_in_leaf=10,
                         max_bin=256, max_depth=6)
model.fit(X, y==0, 
          eval_sets = [
              {'X': X, 'y': y==0, 'sample_weight': w}, 
              {'X': X_test, 'y': y_test==0}
          ]
)

Iter 0; Sample 0, F1_score = 0.0;  Sample 1, F1_score = 0.0; 
Iter 100; Sample 0, F1_score = 0.11107754707336426;  Sample 1, F1_score = 0.09315441462465965; 
Iter 200; Sample 0, F1_score = 0.5074315667152405;  Sample 1, F1_score = 0.4699369521759188; 
Iter 300; Sample 0, F1_score = 0.6575973033905029;  Sample 1, F1_score = 0.6089293096320794; 
Iter 400; Sample 0, F1_score = 0.7355354428291321;  Sample 1, F1_score = 0.6813627254509018; 
Iter 500; Sample 0, F1_score = 0.7783896327018738;  Sample 1, F1_score = 0.7218654529584139; 
Iter 600; Sample 0, F1_score = 0.8057605028152466;  Sample 1, F1_score = 0.7515475179026581; 
Iter 700; Sample 0, F1_score = 0.8268206119537354;  Sample 1, F1_score = 0.7722701576911633; 
Iter 800; Sample 0, F1_score = 0.841198742389679;  Sample 1, F1_score = 0.7864825154275639; 
Iter 900; Sample 0, F1_score = 0.8558464050292969;  Sample 1, F1_score = 0.8005572324123521; 
Iter 999; Sample 0, F1_score = 0.8671237826347351;  Sample 1, F1_score = 0.8101396792550439

In [14]:
%%time
model = GradientBoosting('multiclass', MultiF1Score(average='macro'), 
                         ntrees=5, lr=.01, verbose=100, es=0, lambda_l2=1,
                         subsample=.8, colsample=.8, min_data_in_leaf=10, target_splitter='Single',
                         max_bin=256, max_depth=6)
model.fit(X, y, sample_weight=w,
          eval_sets = [
              {'X': X, 'y': y, 'sample_weight': w
              }, 
              {'X': X_test, 'y': y_test}
          ]
         )

Iter 0; Sample 0, score = 0.4537686115771251;  Sample 1, score = 0.44276431290263557; 
Iter 4; Sample 0, score = 0.5768719064352923;  Sample 1, score = 0.5572464068843448; 
CPU times: user 3.91 s, sys: 49.4 ms, total: 3.96 s
Wall time: 1.45 s


In [15]:
pp = model.predict(X)